# Recommender Code Along

The classic recommender tutorial uses the [movielens data set](https://grouplens.org/datasets/movielens/). It is similar to using the iris or MNIST data set for other algorithms. Let's do a code along to get an idea of how this all works!


Looking for more datasets? Check out: https://gist.github.com/entaroadun/1653794

In [1]:
import findspark
findspark.init('/home/gkouskosv/spark-2.4.5-bin-hadoop2.6/')

In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName('rec').getOrCreate()

With Collaborative filtering we make predictions (filtering) about the interests of a user by collecting preferences or taste information from many users (collaborating). The underlying assumption is that if a user A has the same opinion as a user B on an issue, A is more likely to have B's opinion on a different issue x than to have the opinion on x of a user chosen randomly.

The image below (from Wikipedia) shows an example of collaborative filtering. At first, people rate different items (like videos, images, games). Then, the system makes predictions about a user's rating for an item not rated yet. The new predictions are built upon the existing ratings of other users with similar ratings with the active user. In the image, the system predicts that the user will not like the video.

<img src=https://upload.wikimedia.org/wikipedia/commons/5/52/Collaborative_filtering.gif />

Spark MLlib library for Machine Learning provides a Collaborative Filtering implementation by using Alternating Least Squares. The implementation in MLlib has these parameters:

* numBlocks is the number of blocks used to parallelize computation (set to -1 to auto-configure).
* rank is the number of latent factors in the model.
* iterations is the number of iterations to run.
* lambda specifies the regularization parameter in ALS.
* implicitPrefs specifies whether to use the explicit feedback ALS variant or one adapted for implicit feedback data.
* alpha is a parameter applicable to the implicit feedback variant of ALS that governs the baseline confidence in preference observations.

Let's see this all in action!

In [3]:
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS

In [4]:
data = spark.read.csv('movielens_ratings.csv', inferSchema=True, header=True)

In [5]:
data.head()

Row(movieId=2, rating=3.0, userId=0)

In [6]:
data.describe().show()

+-------+------------------+------------------+------------------+
|summary|           movieId|            rating|            userId|
+-------+------------------+------------------+------------------+
|  count|              1501|              1501|              1501|
|   mean| 49.40572951365756|1.7741505662891406|14.383744170552964|
| stddev|28.937034065088994| 1.187276166124803| 8.591040424293272|
|    min|                 0|               1.0|                 0|
|    max|                99|               5.0|                29|
+-------+------------------+------------------+------------------+



We can do a split to evaluate how well our model performed, but keep in mind that it is very hard to know conclusively how well a recommender system is truly working for some topics. Especially if subjectivity is involved, for example not everyone that loves star wars is going to love star trek, even though a recommendation system may suggest otherwise.

In [7]:
# Smaller dataset, so we will use 0.8 and 0.2

(training, test) = data.randomSplit([0.8,0.2])

In [9]:
# Build the recommendation model using ALS on the training data

als = ALS(maxIter=5, regParam=0.01, 
          userCol='userId',
          itemCol='movieId',
          ratingCol='rating')

In [10]:
model = als.fit(training)

Now let's see hwo the model performed!

In [11]:
# Evaluate the model by computing the RMSE on the test data
predictions = model.transform(test)

In [12]:
predictions.show()

+-------+------+------+----------+
|movieId|rating|userId|prediction|
+-------+------+------+----------+
|     31|   4.0|    12| 1.0356898|
|     31|   1.0|    19| 1.8725853|
|     85|   2.0|    20| 1.5171323|
|     85|   1.0|     4| 2.4137852|
|     85|   1.0|    29|0.17230467|
|     85|   3.0|    21| 2.7785604|
|     65|   1.0|    28| 2.0793712|
|     65|   1.0|     4|-1.2428819|
|     53|   3.0|    13| 2.6816118|
|     53|   3.0|    14| 3.9823775|
|     78|   1.0|    28| 0.5908461|
|     78|   1.0|    13| 0.9537473|
|     78|   1.0|    17|0.77822745|
|     78|   1.0|     4| 1.2514114|
|     34|   1.0|    15| 1.3863351|
|     34|   1.0|     0|0.57834846|
|     81|   5.0|    28| 3.2513413|
|     81|   1.0|    16| 1.8957319|
|     81|   1.0|     7|  2.234981|
|     81|   3.0|    18| 0.6924274|
+-------+------+------+----------+
only showing top 20 rows



In [14]:
evaluator = RegressionEvaluator(metricName='rmse',
                               labelCol='rating',
                               predictionCol='prediction')

rmse = evaluator.evaluate(predictions)

In [15]:
print(f'RMSE = {rmse}')

RMSE = 1.7531093610995354


The RMSE described our error in terms of the stars rating column.

So now that we have the model, how would you actually supply a recommendation to a user?

The same way we did with the test data! For example:

In [17]:
single_user = test.filter(test['userId'] == 11).select(['movieId','userId'])

In [19]:
# User had 10 rating in the test dataset 
# Realistically this should be some of hold out set
single_user.show()

+-------+------+
|movieId|userId|
+-------+------+
|     12|    11|
|     16|    11|
|     19|    11|
|     20|    11|
|     23|    11|
|     30|    11|
|     40|    11|
|     51|    11|
|     66|    11|
|     88|    11|
|     90|    11|
+-------+------+



In [20]:
recommendations = model.transform(single_user)

In [23]:
recommendations.orderBy('prediction', ascending=0).show()

+-------+------+-----------+
|movieId|userId| prediction|
+-------+------+-----------+
|     88|    11|  5.5825043|
|     19|    11|    4.38036|
|     12|    11|   4.314304|
|     51|    11|  4.0945196|
|     40|    11|  2.1895218|
|     66|    11|  1.4908937|
|     23|    11| 0.46007597|
|     90|    11| -0.5770844|
|     30|    11|-0.59059423|
|     20|    11| -1.5969573|
|     16|    11| -1.7011678|
+-------+------+-----------+

